# Code for training the holistic model

## Import Libraries 


In [1]:
import tensorflow as tf
import numpy as np
import os
from datetime import datetime

from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

## Setup Variables

In [2]:
FOLDER_NAME = 'dataset'
ALL_CLASSES = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [3]:
# Create label map, representing each class as a number
label_map = {}
for (root, folders, files) in os.walk(FOLDER_NAME):
    for foldername in folders:
        if foldername in ALL_CLASSES:
            label_map[foldername] = ALL_CLASSES.index(foldername)

print(label_map)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [4]:
# Get all datset data with its label and put it in a list
sequence, label = [], []
target_length = 14
for (root, folders, files) in os.walk(FOLDER_NAME):
    total_file = 0
    for filename in files:
        file_path = os.path.join(os.path.relpath(
            root, FOLDER_NAME), filename)
        if (filename.endswith('.npy') and os.path.split(file_path)[0] in ALL_CLASSES):
            res = np.load(f'{FOLDER_NAME}/{file_path}')
            for _ in range(target_length-res.shape[0]):
                res = np.vstack((res, res[-1, :]))
            res = res[:, -126:]
            sequence.append(np.array(res))
            label.append(label_map[os.path.basename(root[-1])])
            total_file += 1
    print(f"Total files: {total_file} --- {root}")

print(np.array(sequence).shape)
print(np.array(label).shape)

Total files: 0 --- dataset
Total files: 40 --- dataset\a
Total files: 24 --- dataset\b
Total files: 20 --- dataset\c
Total files: 20 --- dataset\d
Total files: 19 --- dataset\e
Total files: 30 --- dataset\f
Total files: 20 --- dataset\g
Total files: 30 --- dataset\h
Total files: 30 --- dataset\i
Total files: 22 --- dataset\j
Total files: 20 --- dataset\k
Total files: 35 --- dataset\l
Total files: 22 --- dataset\m
Total files: 21 --- dataset\n
Total files: 30 --- dataset\o
Total files: 19 --- dataset\p
Total files: 20 --- dataset\r
Total files: 21 --- dataset\s
Total files: 10 --- dataset\t
Total files: 29 --- dataset\u
Total files: 20 --- dataset\v
Total files: 20 --- dataset\w
Total files: 10 --- dataset\x
Total files: 10 --- dataset\y
Total files: 60 --- dataset\z
(602, 14, 126)
(602,)


In [6]:
# Dataset duplication if necessary
n = 2
sequence = np.concatenate([sequence] * n, axis=0)
label = np.concatenate([label] * n, axis=0)


print(np.array(sequence).shape)
print(np.array(label).shape)

(2408, 14, 126)
(2408,)


In [7]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Training Data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sequence), tf.keras.utils.to_categorical(
    np.array(label).astype(int), num_classes=np.array(ALL_CLASSES).shape[0], dtype='float32'), test_size=0.2)

print(X_train.shape, X_test.shape)


(1926, 14, 126) (482, 14, 126)


In [9]:
pose_end = 33 * 4  # 132
face_end = pose_end + 468 * 3  # 1536
lh_end = face_end + 21 * 3  # 1599
rh_end = lh_end + 21 * 3  # 1662

# Separate the data
other_data = X_train[:, :, :face_end]  # Include pose and face
hands_data = X_train[:, :, face_end:]  # Include left and right hand

print("Other data shape:", other_data.shape)
print("Hands data shape:", hands_data.shape)



Other data shape: (1926, 14, 126)
Hands data shape: (1926, 14, 0)


In [10]:
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")

log_dir = os.path.join('Logs', current_time)
tb_callback = TensorBoard(log_dir=log_dir)

print(log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True,
          activation='tanh', input_shape=(14, 126)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.array(ALL_CLASSES).shape[0], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

Logs\20240627-194439


In [11]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])


Epoch 1/200
61/61 [==============================] - 10s 15ms/step - loss: 2.9041 - categorical_accuracy: 0.1241
Epoch 2/200
61/61 [==============================] - 1s 13ms/step - loss: 2.2348 - categorical_accuracy: 0.2586
Epoch 3/200
61/61 [==============================] - 1s 12ms/step - loss: 1.9277 - categorical_accuracy: 0.2908
Epoch 4/200
61/61 [==============================] - 1s 21ms/step - loss: 1.7783 - categorical_accuracy: 0.3032
Epoch 5/200
61/61 [==============================] - 1s 13ms/step - loss: 1.6413 - categorical_accuracy: 0.3593
Epoch 6/200
61/61 [==============================] - 2s 25ms/step - loss: 1.5452 - categorical_accuracy: 0.3884
Epoch 7/200
61/61 [==============================] - 1s 13ms/step - loss: 1.5020 - categorical_accuracy: 0.4050
Epoch 8/200
61/61 [==============================] - 1s 14ms/step - loss: 1.3841 - categorical_accuracy: 0.4476
Epoch 9/200
61/61 [==============================] - 1s 13ms/step - loss: 1.3225 - categorical_accuracy

KeyboardInterrupt: 

In [12]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 14, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 14, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 26)                858       
                                                                 
Total params: 204,218
Trainable params: 204,218
Non-trai

In [13]:
res = model.predict(X_test)

16/16 [==============================] - 1s 12ms/step


In [14]:


# Assume X_test and y_test are your test dataset
# Make predictions
y_pred = model.predict(X_test)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)  # if y_test is one-hot encoded

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred_classes)

# Calculate precision, recall, and F1 score for each class
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Detailed classification report
report = classification_report(y_true, y_pred_classes)
print(report)


16/16 [==============================] - 0s 14ms/step
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        29
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        22
           3       1.00      1.00      1.00        19
           4       1.00      1.00      1.00        14
           5       1.00      1.00      1.00        23
           6       1.00      1.00      1.00        17
           7       1.00      1.00      1.00        23
           8       1.00      1.00      1.00        23
           9       1.00      1.00      1.00        11
          10       1.00      1.00      1.00        21
          11       1.00      1.00      1.00        34
          12       1.00      1.00      1.00         9
          13       1.00      1.00      1.00        16
          14       1.00      1.00      1.00        26
          15       1.00   

In [15]:
actions = np.array(ALL_CLASSES)
testing = 17
print(actions[np.argmax(res[testing])], actions[np.argmax(y_test[testing])])

k k


In [16]:
model.save('action.h5')

In [17]:
model.load_weights('action.h5')

In [18]:
yhat = model.predict(X_test)

16/16 [==============================] - 0s 14ms/step


In [19]:
ytrue = np.argmax(y_test, axis=1).tolist()


In [20]:
yhat = np.argmax(yhat, axis=1).tolist()

In [21]:

multilabel_confusion_matrix(ytrue, yhat)

array([[[453,   0],
        [  0,  29]],

       [[461,   0],
        [  0,  21]],

       [[460,   0],
        [  0,  22]],

       [[463,   0],
        [  0,  19]],

       [[468,   0],
        [  0,  14]],

       [[459,   0],
        [  0,  23]],

       [[465,   0],
        [  0,  17]],

       [[459,   0],
        [  0,  23]],

       [[459,   0],
        [  0,  23]],

       [[471,   0],
        [  0,  11]],

       [[461,   0],
        [  0,  21]],

       [[448,   0],
        [  0,  34]],

       [[473,   0],
        [  0,   9]],

       [[466,   0],
        [  0,  16]],

       [[456,   0],
        [  0,  26]],

       [[472,   0],
        [  0,  10]],

       [[465,   0],
        [  0,  17]],

       [[465,   0],
        [  0,  17]],

       [[473,   0],
        [  0,   9]],

       [[456,   0],
        [  0,  26]],

       [[469,   0],
        [  0,  13]],

       [[460,   0],
        [  0,  22]],

       [[473,   0],
        [  0,   9]],

       [[474,   0],
        [  0, 

In [22]:
accuracy_score(ytrue, yhat)

1.0